## Serve trained model on your local machine

### Prepare infrastructure in minikube

#### ensure starting from scratch

In [1]:
!minikube delete --all --purge


🔥  Deleting "minikube" in docker ...
🔥  Removing /home/dwarszawski/.minikube/machines/minikube ...
💀  Removed all traces of the "minikube" cluster.
🔥  Successfully deleted all profiles
💀  Successfully purged minikube directory located at - [/home/dwarszawski/.minikube]


#### check minikube status

In [43]:
!minikube status

🤷  There is no local cluster named "minikube"
👉  To fix this, run: "minikube start"


#### start minikube

In [1]:
!minikube start --kubernetes-version v1.17.0

😄  minikube v1.11.0 on Ubuntu 20.04
✨  Using the docker driver based on existing profile
🆕  Kubernetes 1.18.3 is now available. If you would like to upgrade, specify: --kubernetes-version=v1.18.3
🆕  Kubernetes 1.18.3 is now available. If you would like to upgrade, specify: --kubernetes-version=v1.18.3
🆕  Kubernetes 1.18.3 is now available. If you would like to upgrade, specify: --kubernetes-version=v1.18.3
🆕  Kubernetes 1.18.3 is now available. If you would like to upgrade, specify: --kubernetes-version=v1.18.3
👍  Starting control plane node minikube in cluster minikube
🔄  Restarting existing docker container for "minikube" ...
🎉  minikube 1.13.1 is available! Download it: https://github.com/kubernetes/minikube/releases/tag/v1.13.1
💡  To disable this notice, run: 'minikube config set WantUpdateNotification false'

🐳  Preparing Kubernetes v1.17.0 on Docker 19.03.2 ...
    ▪ kubeadm.pod-network-cidr=10.244.0.0/16
🔎  Verifying Kubernetes components...
🌟  Enabled addons: ambassador, defaul

In [45]:
!minikube status


minikube
type: Control Plane
host: Running
kubelet: Running
apiserver: Running
kubeconfig: Configured



#### switch kubectl context to use minikube cluster

In [4]:
!kubectl config current-context

minikube


#### create namespace to be used for installing seldon operator

In [5]:
!helm list  --namespace seldon

NAME       	NAMESPACE	REVISION	UPDATED                                 	STATUS  	CHART                     	APP VERSION
seldon-core	seldon   	1       	2020-09-25 23:56:47.572732655 +0200 CEST	deployed	seldon-core-operator-1.2.3	           


In [3]:
!kubectl create namespace seldon


namespace/seldon created


In [4]:
!helm install seldon-core seldon-core-operator \
    --repo https://storage.googleapis.com/seldon-charts \
    --set usageMetrics.enabled=true \
    --set ambassador.enabled=true \
    --namespace seldon

NAME: seldon-core
LAST DEPLOYED: Mon Sep 28 09:58:40 2020
NAMESPACE: seldon
STATUS: deployed
REVISION: 1
TEST SUITE: None


#### Enable ambassador in minikube

In [5]:
!minikube addons enable ambassador

🌟  The 'ambassador' addon is enabled


#### Required by Ambassador to expose endpoint

In [ ]:
#!minikube tunnel

#### Secrets to authenticate for artifacts available in GCP storage

In [41]:
#!gcloud auth activate-service-account --key-file spark-bigdl-845ec4674852.json
!kubectl create secret generic user-gcp-sa --namespace seldon --from-file=gcloud-application-credentials.json=spark-bigdl-845ec4674852.json

Error from server (AlreadyExists): secrets "user-gcp-sa" already exists


In [56]:
#!kubectl delete secret user-gcp-sa --namespace seldon

secret "user-gcp-sa" deleted


#### Service account with secrets assigned

In [7]:
!kubectl apply -f service-account.yaml --namespace seldon



serviceaccount/user-gcp-sa created


In [37]:
#!kubectl delete sa user-gcp-sa --namespace seldon

serviceaccount/user-gcp-sa configured


### Serve your model and explainer with REST API

In [8]:
!kubectl apply -f seldon-deployment.yaml

seldondeployment.machinelearning.seldon.io/dssconf2020 created


#### Check ambassador endpoint

In [2]:
!kubectl get services --namespace ambassador

NAME                          TYPE           CLUSTER-IP      EXTERNAL-IP   PORT(S)                      AGE
ambassador                    LoadBalancer   10.96.26.63     10.96.26.63   80:32180/TCP,443:30424/TCP   37h
ambassador-admin              ClusterIP      10.96.248.223   <none>        8877/TCP                     37h
ambassador-operator-metrics   ClusterIP      10.96.25.232    <none>        8383/TCP,8686/TCP            37h


#### Verify if prediction endpoint exposed with sample input

In [9]:
#!curl -v localhost:9000/api/v1.0/predictions -H "Content-Type: application/json" -d '{"data": {"ndarray": [[4.0, 0.166, 0.986, 0.0, 2.0, 0.0, 1.0]]}}'

!curl http://10.96.26.63/seldon/seldon/dssconf2020/api/v1.0/predictions -H \
    "Content-Type: application/json" \
    -d '{"data": {"ndarray": [[17.75, -0.89100584, 0.45399184, 0.0, 5.0, 1.0, 1.0]]}}'

{"data":{"names":[],"ndarray":[0.014660171157941985]},"meta":{}}


#### Get explanation for prediction

In [3]:
!curl  http://10.96.26.63/seldon/seldon/dssconf2020-explainer/default/api/v1.0/explain \
    -H "Content-Type: application/json" \
    -d '{"data": {"ndarray": [[17.75, -0.89100584, 0.45399184, 0.0, 5.0, 1.0, 1.0]]}}' | \
    jq  '.'


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 40364  100 40287  100    77  26314     50  0:00:01  0:00:01 --:--:-- 26364
{
  "names": [
    "passenger_count <= 1.00",
    "weekend <= 1.00",
    "sin_pickup_hour <= -0.52",
    "pickup_hour <= 18.30",
    "weekday <= 5.00",
    "cos_pickup_hour <= 0.50",
    "night_hours <= 0.00"
  ],
  "precision": 0,
  "coverage": 0.0748,
  "raw": {
    "feature": [
      6,
      5,
      1,
      0,
      4,
      1,
      0,
      2,
      0,
      2,
      2,
      1,
      6,
      3
    ],
    "mean": [
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0
    ],
    "precision": [
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0
    ],
    "coverage": [
      0.8514,
      0.8514,
      

In [32]:
!kubectl get pods --namespace seldon

*   Trying 127.0.0.1:80...
* TCP_NODELAY set
* connect to 127.0.0.1 port 80 failed: Connection refused
* Failed to connect to localhost port 80: Connection refused
* Closing connection 0
curl: (7) Failed to connect to localhost port 80: Connection refused


In [ ]:
!kubectl get pods --namespace seldo

In [23]:
!helm delete seldon-core --namespace seldon
!kubectl delete crds --all


seldondeployment.machinelearning.seldon.io/dssconf2020 created


In [ ]:
!kubectl get pods --namespace seldo

#### Clean up minikube

In [ ]:
!helm delete seldon-core --namespace seldon
!kubectl delete crds --all